# Metacritic Scraper for PlayStation Posts 🎮

**https://www.metacritic.com/robots.txt**

In [1]:
import requests as re
import json as json 
import os as os
from bs4 import BeautifulSoup as bs 
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import pandas as pd
import time

page number to go back one year is 90 (~ 15 ish dec 2022)

In [3]:
def article_scraper(page_number):
    url = f'https://www.metacritic.com/browse/game/all/all/all-time/new/?releaseYearMin=1958&releaseYearMax=2023&platform=ps5&platform=ps4&page={page_number}'
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"}  # metacritic blocks default user agent so I needed to tell it it's me 🤷‍♂️
    response = re.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Status code: {response.status_code}")  # since they block default user agents, I needed to check to make sure the request went through 👺
    soup = bs(response.content, 'html.parser')

    games = soup.find_all('div', class_='c-finderProductCard')
    data = []
    for game in games:
        link_element = game.find('a', class_='c-finderProductCard_container')
        title_element = game.find('h3', class_='c-finderProductCard_titleHeading')
        date_element = game.find('div', class_='c-finderProductCard_meta').find('span')
        description_element = game.find('div', class_='c-finderProductCard_description').find('span')

        link = f"https://www.metacritic.com{link_element['href']}" if link_element else None
        title = title_element.get_text(strip=True) if title_element else None
        date = date_element.get_text(strip=True) if date_element else None
        description = description_element.get_text(strip=True) if description_element else None
        
        data.append({'Title': title, 'Release Date': date, 'Description': description, 'Game Link': link})
    return data


N = 90  # Manually adjust number of pages to scrape
all_data = []
for page in range(1, N + 1):
    print(f"Scraping page {page}...")
    all_data.extend(article_scraper(page))

df = pd.DataFrame(all_data)

print(df)


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Scraping page 37...
Scraping page 38...
Scraping page 39...
Scraping page 40...
Scraping page 41...
Scraping page 42...
Scraping page 43...
Scraping page 44...
Scraping page 45...
Scraping page 46...
Scraping page 47...
Scraping page 48...
Scraping page 49...
Scraping page 50...
Scraping 

In [4]:
from itables import show 

show(df)

In [5]:
# saving it just in case we want to play with it later 
# df.to_csv('RAW_metacritic_ps_titles.csv')

In [44]:
def comments_spider(df_batch):
    driver = webdriver.Chrome()
    total_games = len(df)
    comments_data = []

    for index, (game_index, game_row) in enumerate(df_batch.iterrows(), start=1): 

        game_link = game_row['Game Link']
        game_title = game_link.split('/')[-2].replace('-', ' ')

        try:
            print(f"{index + batch_start - 1}/{total_games}: Processing {game_title} at {game_link}")
            driver.get(game_link)

            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(3)

                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            soup = bs(driver.page_source, 'html.parser')
            critic_reviews = soup.find_all('div', class_='c-reviewsSection_criticReviews')
            user_reviews = soup.find_all('div', class_='c-reviewsSection_userReviews')
            num_critic_comments = len([element for element in critic_reviews if element.find('div', class_='c-siteReview_quote')])
            num_user_comments = len([element for element in user_reviews if element.find('div', class_='c-siteReview_quote')])
            num_comments = num_critic_comments + num_user_comments
            
            print(f"🔎 -- Found {num_comments} reviews for {game_title} -- 🔍")

            for review in critic_reviews + user_reviews:
                console_element = review.find('a', class_='c-siteReview_platform')
                console_type = console_element.get_text(strip=True) if console_element else 'null'

                commenter_name_element = review.find('a', class_='c-siteReviewHeader_username')
                if commenter_name_element:
                    commenter_name = commenter_name_element.get_text(strip=True)
                else:
                    commenter_name_element = review.find('a', class_='c-siteReviewHeader_publicationName')
                    commenter_name = commenter_name_element.get_text(strip=True) if commenter_name_element else 'null'

                reviewer_type = 'Critic' if review in critic_reviews else 'User'

                comment_element = review.find('div', class_='c-siteReview_quote')
                if comment_element and comment_element.span:
                    comment_text = comment_element.span.get_text(strip=True)
                    # print(f'Comment Text: {comment_text}')
                else:
                    comment_text = 'null' 
                    # print('No comment found')  

                date_element = review.find('div', class_='c-siteReviewHeader_reviewDate')
                date_commented = date_element.get_text(strip=True) if date_element else 'null'

                score_element = review.find('div', class_='c-siteReviewScore')
                score_number = score_element.span.get_text(strip=True) if score_element and score_element.span else 'null'

                comment_data = {
                        'Game Title': game_title,
                        'Console': console_type,
                        'Commenter Name': commenter_name,
                        'Reviewer Type': reviewer_type,
                        'Comment': comment_text,
                        'Date Commented': date_commented,
                        'Score Number': score_number
                    }
                comments_data.append(comment_data)
                
                # print(f'Appending data: {comment_data}') 

        except Exception as e:
            print(f'Error processing {game_title} at {game_link}: {e}')
            continue 

    driver.quit()
    comments_df = pd.DataFrame(comments_data)
    return comments_df

batch_start = 2156 # --- 🟧 Manually adjust for batching. Batch start isn't 'how many' it's to 'which index' 🟧 note to self: start at end of last batch ---
batch_size = 10  # --- 🟧 Manually adjust for batching. 🟧 
df_batch = df.iloc[batch_start - 1:batch_start - 1 + batch_size]

raw_comments_df = comments_spider(df_batch)
print(raw_comments_df)

2156/2160: Processing richman 10 at https://www.metacritic.com/game/richman-10/
🔎 -- Found 1 reviews for richman 10 -- 🔍
2157/2160: Processing mahjong 2016 at https://www.metacritic.com/game/mahjong-2016/
🔎 -- Found 1 reviews for mahjong 2016 -- 🔍
2158/2160: Processing beholder 3 at https://www.metacritic.com/game/beholder-3/
🔎 -- Found 4 reviews for beholder 3 -- 🔍
2159/2160: Processing cassiodora at https://www.metacritic.com/game/cassiodora/
🔎 -- Found 1 reviews for cassiodora -- 🔍
2160/2160: Processing grime at https://www.metacritic.com/game/grime/
🔎 -- Found 4 reviews for grime -- 🔍
      Game Title        Console         Commenter Name Reviewer Type  \
0     richman 10             PC               Gamersky        Critic   
1     richman 10           null                   null          User   
2   mahjong 2016  Xbox Series X             TheXboxHub        Critic   
3   mahjong 2016           null                   null          User   
4     beholder 3             PC  Movies Game

---
---
---

### Lab Testing Area 🔍👨‍⚕️

In [36]:
from itables import show

In [45]:
print(raw_comments_df.shape)

(14, 7)


In [46]:
show(raw_comments_df)

In [47]:
# run if all is good 
# raw_comments_df.to_csv('RAW_metacritic_ps_comments_batch2156-2160.csv')